# Project Name: Salary Prediction From Scratch To Deployment

## Part 1 : Web Scrapping Using Selenium

In [4]:
## Import the libraries

from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import pandas as pd
import time

In [5]:
## set a path of chromedriver and open maximized window

driver=webdriver.Chrome(executable_path="C:/ChromeDriver/chromedriver")
driver.maximize_window()
time.sleep(5)

In [6]:
## Login to glassdoor website 

def web_login(web_link,user_name,pass_word):
    
    ''' make get request on glassdoor website and 
    enter the username and password to login'''
    
    driver.get(web_link)
    time.sleep(5)
    username=driver.find_element_by_name("username")
    username.send_keys(user_name)
    password=driver.find_element_by_name("password")
    password.send_keys(pass_word)
    password.submit()


## define the variables for login
web_link="https://www.glassdoor.co.in/profile/login_input.htm?userOriginHook=HEADER_SIGNIN_LINK"
user_name="Shubhkalpatil122@gmail.com"
pass_word="Shubham@123"


## call the fucntion to login
web_login(web_link,user_name,pass_word)
time.sleep(5)

In [14]:
## move to salary search page

def seach_job(job_title):
    
    ''' Click on salaries on homepage and enter the job title
    and city name in which you are looking for details of salaries'''
    
    salaries=driver.find_element_by_link_text("Salaries")
    salaries.click()
    job_name=driver.find_element_by_name("sc.keyword")
    job_name.send_keys(job_title)
    job_name.submit()


## call function : seach_job
job_title="Data Scientist"
seach_job(job_title)
time.sleep(5)

In [16]:
## Extract the main link of each company

def extract_all_job_links(required_job_total,a):
    
    ''' from all pages, extract the link of each company, so that 
    we can individually visit each page in further steps'''
    
    while len(jobs)<=required_job_total:
        job_link=driver.find_elements_by_xpath('//a[contains(@href,"Salary/")]')
        for i in job_link[1::3]:
            jobs.append(i.get_attribute("href"))
        a=a+1
        
        print(a,float(len(jobs)))
        driver.get("https://www.glassdoor.co.in/Salaries/bangalore-data-scientist-salary-SRCH_IL.0,9_IM1091_KO10,24_IP"
                   +str(a+1)+".htm?clickSource=searchBtn")


## set variable value and call the function:" extract_all_job_links"
required_job_total=12    ## We change this number as per requirnment
a=0
jobs=[]
role=[]
extract_all_job_links(required_job_total,a)
print("Total number of jobs extracted are",len(jobs),"and total required jobs were",required_job_total)

1 27.0
Total number of jobs extracted are 27 and total required jobs was 15


In [17]:
## Information Extraction From Salary Page

def salary_page():
    
    ''' Here we will enter to each company page to extact all required detail from each subpage,
    there is multiple instance where information is missing hence used try and except for smooth
    coding operation. Also missing details will be taken as "NA"'''
    
    ## Extract the Job Title
    Role=driver.find_element_by_tag_name("h1").text
    Role1.append(Role)
    
    '''From salary page extact Basic,additional,average,min and max salary details'''
    
    ## Basic Salary
    try:
        base_sal=driver.find_element_by_xpath("//div[starts-with(@class,'d-flex')]/h2").text
    except:
        base_sal1.append("NA")
    else:
        base_sal1.append(base_sal)
        
    ## Additional Salary
    try:
        add_pay=driver.find_element_by_class_name("mx-xxsm").text
    except:
        add_pay1.append("NA")
    else:
        add_pay1.append(add_pay)
        
    ## Average Salary   
    try:
        avg_pay=driver.find_element_by_class_name("css-5j5djr").text
    except:
        avg_pay1.append("NA")        
    else:
        avg_pay1.append(avg_pay)
        
    ## Minimumn and Maximum Salary   
    try:
        min_max_sal=driver.find_element_by_xpath("//div[contains(@class,'my-xxsm')]").text.split("\n")   
    except:
        max_sal1.append("NA")
        min_sal1.append("NA")
    else:
        max_sal1.append(min_max_sal[1])
        min_sal1.append(min_max_sal[0])
    
    ## Average Salary from text   
    try:
        Avg_sal=driver.find_element_by_class_name("css-1aphn3v").text
    except:
        avg_pay_num.append("NA")
        avg_pay_text.append("NA")
    else:
        try:
            Avg_full_text=Avg_sal.split(".")[0]
            Avg_num=Avg_sal[Avg_sal.index("₹"):Avg_sal.index(".")]
        except:
            avg_pay_num.append("NA")
            avg_pay_text.append("NA")
        else:
            avg_pay_num.append(Avg_num)
            avg_pay_text.append(Avg_full_text)
            


In [18]:
## Information Extraction Interview Page

def interview_page():
    
    ''' Here we will extract the percentage details of inerview experience 
    and Hiring mediums'''
    
    label=["Positive","Negative","Neutral","Online","Campus","Refferal"]
    interview=[]
    driver.find_element_by_class_name("interviews").click()
    
    try:
        int_difficulty=driver.find_element_by_class_name("css-155sv15").text
        int_alldetail=driver.find_elements_by_class_name("ml-auto")
    except:
        interviews=dict(zip(label,["NA" for i in range(len(label))]))
        interview1.append(interviews)
    else:
        for i in int_alldetail:
            interview.append(i.text)
        interviews=dict(zip(label,interview))
        interview1.append(interviews)



In [19]:
## Information Extraction From Company Overview page

def Company_Overview_page():
    
    ''' from this page aim is to extrat the details of of company like locations, size, founding year etc.
    al this information will stored in dict, which will further appended to comp_overview1, wherever details
    is missing, take it as "NA"'''
    
    try:
        labels=["Headquarter","Size","Founded","Type","Industry","Revenue"]
        driver.find_element_by_class_name("icon-bullseye-select").click()
        driver.find_element_by_link_text("Company Overview").click()
    except:
        rating1.append("NA")
        comp_overview=dict(zip(labels,["NA" for i in range(len(labels))]))
        comp_overview1.append(comp_overview)
    else:
        try:
            Headquarter=driver.find_elements_by_class_name("css-dwl48b")[:6]
        except:
            comp_overview=dict(zip(labels,["NA" for i in range(len(labels))]))
            comp_overview1.append(comp_overview)
        else:
            overview=[i.text for i in Headquarter]
            comp_overview=dict(zip(labels,overview))
            comp_overview1.append(comp_overview)
        try:
            rating=driver.find_element_by_class_name("eky1qiu0").text
        except:
            rating1.append("NA")
        else:
            rating1.append(rating)



In [20]:
## Call All Above Function Together

def all_features_extract(jobs_list):
    
    ''' make a request on each company page and further call 
    predefined functions to visit each subpage to extract details'''
    
    for i in jobs_list:
        driver.get(i) 
        try:
            Employer=driver.find_element_by_class_name("tightAll").text
        except:
            continue
        else:
            Employer1.append(Employer)
            salary_page()
            interview_page()      
            Company_Overview_page()
            print(Employer)


## Define variables and call function "all_features_extract"
base_sal1=[]
add_pay1=[]
avg_pay1=[]
avg_pay_num=[]
avg_pay_text=[]
max_sal1=[]
min_sal1=[]
Employer1=[]
Role1=[]
interview1=[]
comp_overview1=[]
rating1=[]

all_features_extract(jobs[:])

Mu Sigma
IBM
Impact Analytics
Accenture
Infosys
Embibe
Cognizant Technology Solutions
Anheuser-Busch InBev
Capgemini


## Convert Scrapped Data To Dataframe

In [22]:
##  Convert Scrapped Information Of Salaries Into Dataframe

df_Salary=pd.DataFrame({"Employer":Employer1,"Job_Title":Role1,"Basic_Pay":base_sal1,"Additional_Pay":add_pay1,"Avg_Pay":avg_pay1,"Avg_Pay_Num":avg_pay_num,"Avg_Pay_Text":avg_pay_text,"Max_Pay":max_sal1,"Min_Pay":min_sal1,"Rating":rating1})
df_Salary.head()

,Employer,Job_Title,Basic_Pay,Additional_Pay,Avg_Pay,Avg_Pay_Num,Avg_Pay_Text,Max_Pay,Min_Pay,Rating
0,Mu Sigma,Mu Sigma Data Scientist Salaries in Bangalore ...,"₹ 6,19,333","₹ 1,18,566","₹ 6,83,478","₹6,19,333",The typical Mu Sigma Data Scientist salary is ...,"₹4,735K",₹298K,3.3
1,IBM,IBM Data Scientist Salaries in Bangalore Area,"₹ 11,23,120","₹ 43,138","₹ 11,23,120","₹11,23,120","The typical IBM Data Scientist salary is ₹11,2...","₹3,768K",₹292K,3.9
2,Impact Analytics,Impact Analytics Data Scientist Salaries in Ba...,"₹ 6,54,456","₹ 90,798","₹ 7,17,428","₹6,54,456",The typical Impact Analytics Data Scientist sa...,₹909K,₹520K,4.0
3,Accenture,Accenture Data Scientist Salaries in Bangalore...,"₹ 8,80,893","₹ 1,59,128","₹ 8,80,893","₹8,80,893",The typical Accenture Data Scientist salary is...,"₹3,000K",₹420K,4.0
4,Infosys,Infosys Data Scientist Salaries in Bangalore Area,"₹ 9,84,600","₹ 89,245","₹ 10,31,705","₹9,84,600",The typical Infosys Data Scientist salary is ₹...,"₹3,249K",₹351K,3.8


In [23]:
##  Convert Scrapped Information Of Inteviews Into Dataframe

df_interview=pd.DataFrame(interview1)
df_interview.head()

,Positive,Negative,Neutral,Online,Campus,Refferal
0,70%,12%,19%,77%,12%,5%
1,69%,12%,19%,43%,22%,14%
2,48%,32%,19%,38%,38%,10%
3,70%,13%,17%,37%,29%,12%
4,77%,9%,15%,54%,22%,12%


In [24]:
## Convert Scrapped Information Of Company Overviews To Dataframe

df_overview=pd.DataFrame(comp_overview1)
df_overview.head()

,Headquarter,Size,Founded,Type,Industry,Revenue
0,"Northbrook, IL",1001 to 5000 Employees,2004,Company - Private,Consulting,Unknown / Non-Applicable
1,"Bangalore, India",10000+ Employees,1911,Company - Public (IBM),IT Services,$10+ billion (USD)
2,"New York, NY",201 to 500 Employees,2015,Company - Private,Enterprise Software & Network Solutions,Unknown / Non-Applicable
3,"Dublin, Ireland",10000+ Employees,1989,Company - Public (ACN),Consulting,$10+ billion (USD)
4,"Bangalore, India",10000+ Employees,1981,Company - Public (INFY),IT Services,$10+ billion (USD)


## Final Dataset

In [25]:
## combine all dataframes and assign the column names

df=pd.concat([df_overview,df_Salary,df_interview],axis=1)

columns=["Headquarter","Company_Size","Founded","Company_Type",
         "Industry","Revenue","Role","Company_name","Base_Salary",
         "Additional_Pay","Avg_Pay","Avg_Pay_Num","Avg_Pay_Text",
         "Max_Pay","Min_Pay","Rating","Company_rating","Positive_Interview",
         "Negative_Interview","Neutral_interview","Online_Hire","Campus_Hire","Refferal_Hire"]
df.head() 

,Headquarter,Size,Founded,Type,Industry,Revenue,Employer,Job_Title,Basic_Pay,Additional_Pay,...,Avg_Pay_Text,Max_Pay,Min_Pay,Rating,Positive,Negative,Neutral,Online,Campus,Refferal
0,"Northbrook, IL",1001 to 5000 Employees,2004,Company - Private,Consulting,Unknown / Non-Applicable,Mu Sigma,Mu Sigma Data Scientist Salaries in Bangalore ...,"₹ 6,19,333","₹ 1,18,566",...,The typical Mu Sigma Data Scientist salary is ...,"₹4,735K",₹298K,3.3,70%,12%,19%,77%,12%,5%
1,"Bangalore, India",10000+ Employees,1911,Company - Public (IBM),IT Services,$10+ billion (USD),IBM,IBM Data Scientist Salaries in Bangalore Area,"₹ 11,23,120","₹ 43,138",...,"The typical IBM Data Scientist salary is ₹11,2...","₹3,768K",₹292K,3.9,69%,12%,19%,43%,22%,14%
2,"New York, NY",201 to 500 Employees,2015,Company - Private,Enterprise Software & Network Solutions,Unknown / Non-Applicable,Impact Analytics,Impact Analytics Data Scientist Salaries in Ba...,"₹ 6,54,456","₹ 90,798",...,The typical Impact Analytics Data Scientist sa...,₹909K,₹520K,4.0,48%,32%,19%,38%,38%,10%
3,"Dublin, Ireland",10000+ Employees,1989,Company - Public (ACN),Consulting,$10+ billion (USD),Accenture,Accenture Data Scientist Salaries in Bangalore...,"₹ 8,80,893","₹ 1,59,128",...,The typical Accenture Data Scientist salary is...,"₹3,000K",₹420K,4.0,70%,13%,17%,37%,29%,12%
4,"Bangalore, India",10000+ Employees,1981,Company - Public (INFY),IT Services,$10+ billion (USD),Infosys,Infosys Data Scientist Salaries in Bangalore Area,"₹ 9,84,600","₹ 89,245",...,The typical Infosys Data Scientist salary is ₹...,"₹3,249K",₹351K,3.8,77%,9%,15%,54%,22%,12%


## Export Dataset

In [746]:
'''if we want to extract data for multiple cities then repeat above process
for each city and combine all cities to '''

df.to_csv("Glassdoor_scratch.csv")